<a href="https://colab.research.google.com/github/lee191/LSTM/blob/main/data_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.cluster import KMeans
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE


# 컬럼 이름을 직접 지정합니다.
column_names = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins',
    'logged_in', 'num_compromised', 'root_shell', 'su_attempted',
    'num_root', 'num_file_creations', 'num_shells', 'num_access_files',
    'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count',
    'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate',
    'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate',
    'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate',
    'label'
]

df1 = pd.read_csv("data/KDDTrain+.csv", names=column_names)
df2 = pd.read_csv("data/KDDTest+.csv", names=column_names)

newdf = pd.concat([df1,df2], ignore_index=True)

# 0:normal, 1:dos, 2:probe, 3:r2l, 4:u2r
dfdos = newdf.replace(['apache2', 'back', 'land', 'mailbomb', 'neptune', 'pod', 'processtable', 'smurf', 'teardrop', 'udpstorm'], 1)
dfr2l = dfdos.replace(['ftp_write', 'guess_passwd', 'imap', 'multihop', 'named', 'phf', 'sendmail', 'snmpgetattack', 'snmpguess', 'warezmaster', 'worm', 'xlock' ,'xsnoop', 'httptunnel', 'warezclient', 'spy'], 3)
dfu2r = dfr2l.replace(['buffer_overflow', 'loadmodule', 'perl', 'rootkit', 'ps', 'sqlattack', 'xterm'], 4)
dfprobe = dfu2r.replace(['ipsweep', 'mscan', 'nmap', 'portsweep', 'saint', 'satan'], 2)
dfnormal = dfprobe.replace(['normal'], 0)
df_type = dfnormal

condition = (df_type.label == 0) | (df_type.label == 1) #| (df_type.label == 'probe') | (df_type.label == 'r2l') | (df_type.label == 'u2r')
df_dos = df_type[condition]
df_dos = df_dos.reset_index(drop=True)
df_dos

df_dos.value_counts('label')


for columns in df1.columns:
  if df_dos[columns].dtypes == 'object':
    print(columns, '\n종류:', df_dos[columns].unique(), '\n개수:', df_dos[columns].nunique(), '\n')

df_dos_obj = df_dos[['protocol_type','service','flag']]

newdos = df_dos.drop(columns=['label'])
newlabel = df_dos['label']

le = LabelEncoder()

le.fit(newdos['protocol_type'])
protocol = le.transform(newdos['protocol_type'])

le.fit(newdos['service'])
service = le.transform(newdos['service'])

le.fit(newdos['flag'])
flag = le.transform(newdos['flag'])

newdos['protocol_type'] = protocol
newdos['service'] = service
newdos['flag'] = flag

dosle = newdos
yle = newlabel


pd.options.display.float_format = '{:.9f}'.format #지수표현 X
#pd.reset_option('display.float_format')

scaler = sklearn.preprocessing.MinMaxScaler()
np_scaled = scaler.fit_transform(dosle)

xle = pd.DataFrame(np_scaled, columns = dosle.columns)

xle, xle_test, yle, yle_test = train_test_split(xle, yle, test_size=0.2, random_state=2022)

print(xle.shape, yle.shape)
print(xle_test.shape, yle_test.shape)

x=xle.head(1000)
x_test=xle_test.head(200)
y=yle.head(1000)
y_test=yle_test.head(200)

xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
xgb.fit(xle, yle)

yle_preds = xgb.predict(xle_test)
yle_test = yle_test.astype(int)
yle_preds = yle_preds.astype(int)

print('Accuracy: %.2f' % accuracy_score(yle_test, yle_preds))


cm = confusion_matrix(yle_test, yle_preds)

plt.figure(figsize=(16, 9))
sns.heatmap(
    cm,
    annot=True,
    fmt='g',
    xticklabels=['dos', 'normal'],
    yticklabels=['dos', 'normal']
    #xticklabels=['dos', 'normal', 'probe', 'r2l', 'u2r'],
    #yticklabels=['dos', 'normal', 'probe', 'r2l', 'u2r']
)
plt.show()


perm = PermutationImportance(xgb, random_state=1).fit(xle_test, yle_test)
eli5.show_weights(perm, feature_names = xle_test.columns.tolist())

dosoh = pd.get_dummies(newdos, columns=['protocol_type','service','flag'])
yoh = newlabel

dosoh

pd.options.display.float_format = '{:.9f}'.format #지수표현 X
#pd.reset_option('display.float_format')

scaler = sklearn.preprocessing.MinMaxScaler()
np_scaled = scaler.fit_transform(dosoh)

xoh = pd.DataFrame(np_scaled, columns = dosoh.columns)

xoh


xoh, xoh_test, yoh, yoh_test = train_test_split(xoh, yoh, test_size=0.2, random_state=2022)

print(xoh.shape, yoh.shape)
print(xoh_test.shape, yoh_test.shape)

xcol=list(xoh) #rfe 알고리즘 사용 후 컬럼삭제 되므로 저장
xoh

xoh.columns
yoh_test.value_counts()


clf = RandomForestClassifier(n_estimators=10, n_jobs=-1)
rfe = RFE(estimator=clf, n_features_to_select=21, step=1)

yoh = yoh.astype('category')

xnewcol = []
rfe.fit(xoh, yoh)
for i in range(xoh.shape[1]):
  if rfe.support_[i] == True:
    xnewcol.append(xcol[i])
    print('Column: %s' % (xcol[i]))

print(len(xnewcol))

xoh_dos = rfe.transform(xoh)
xoh_dos.shape
xoh_rfedos = pd.DataFrame(xoh_dos, columns=xnewcol)

xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
xgb.fit(xoh, yoh)

yoh_preds = xgb.predict(xoh_test)
yoh_test = yoh_test.astype(int)
yoh_preds = yoh_preds.astype(int)
print('Accuracy: %.2f' % accuracy_score(yoh_test, yoh_preds))


cm = confusion_matrix(yoh_test, yoh_preds)

plt.figure(figsize=(16, 9))
sns.heatmap(
    cm,
    annot=True,
    fmt='g',
    xticklabels=['dos', 'normal'],
    yticklabels=['dos', 'normal']
    #xticklabels=['dos', 'normal', 'probe', 'r2l', 'u2r'],
    #yticklabels=['dos', 'normal', 'probe', 'r2l', 'u2r']
)
plt.show()

perm = PermutationImportance(xgb, random_state=1).fit(xoh_test, yoh_test)
eli5.show_weights(perm, feature_names = xoh_test.columns.tolist())
